# IMPORTS

In [ ]:
# data
import numpy as np
import pandas as pd

# graphics
import plotly.express as px
# import matplotlib as mpl
import matplotlib.pyplot as plt
# import seaborn as sns

# eda
import missingno as msng
import sweetviz as sv

# stats
from pandas.plotting import autocorrelation_plot # test for seasonality
from pandas.plotting import lag_plot # test for autocorrelation
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy import signal

# src
from src.visualization.visualize import plot_line

# PATHS & NAMES

In [ ]:
INTERIM_FILENAME = ...
INTERIM_FOLDER = "../data/interim"
INTERIM_FILEPATH = f"{INTERIM_FOLDER}/{INTERIM_FILENAME}"

REPORT_FOLDER = "../reports"

TARGET = ...

# DATA

In [ ]:
df = ...

In [ ]:
df.info()